In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components.backtest import Backtester
import tofina.components.instrument as instrument
from tofina.extern.pytorch_forecasting import forecastDecoratorDeepAR
from tofina.extern.yfinance import loadHistoricalStockData
import datetime as dt
from pathlib import Path
import shutil
import pandas as pd
import numpy as np
from copy import deepcopy

AAPL = loadHistoricalStockData("AAPL", start="2001-01-01", end="2024-02-14")
NVDA = loadHistoricalStockData("NVDA", start="2001-01-01", end="2024-02-14")

start_date = dt.datetime(2023, 1, 1)
#end_date = dt.datetime(2023, 1, 14)
end_date = dt.datetime(2023, 1, 4)
split_date = start_date - dt.timedelta(days=1)
timestamps = AAPL.index[
    np.logical_and(AAPL.index >= start_date, AAPL.index < end_date)
]
horizon = 20
simulations = 1000
forecaster = forecastDecoratorDeepAR(
    {"AAPL":AAPL, "NVDA":NVDA}, split_date, "./DeapAR_Training", horizon, simulations,max_epochs=1
)
backtester = Backtester(timestamps=timestamps)
backtester.stockDataFromDataFrame(AAPL, "AAPL")
backtester.stockDataFromDataFrame(NVDA, "NVDA")
backtester.registerForecaster(forecaster, ["AAPL", "NVDA"])
backtester.addDeposit(interestRate=0.03/252)
backtester.addIntstrumentToPortfolio("AAPL", "AAPL_Stock", 
            instrument.NonDerivativePayout
        )
backtester.addIntstrumentToPortfolio("NVDA", "NVDA_Stock", 
            instrument.NonDerivativePayout
        )
backtester.addIntstrumentToPortfolio("AAPL", "AAPL_Stock_Short", 
            instrument.NonDerivativePayoutShort
        )
backtester.addIntstrumentToPortfolio("NVDA", "NVDA_Stock_Short", 
            instrument.NonDerivativePayoutShort
        )

def register_option(df_path, ticker):
    df = pd.read_csv(df_path)
    df["Date"] = df["Date"].apply(lambda x: x.replace(" ", ""))
    df["Expiry Date"] = df["Expiry Date"].apply(lambda x: x.replace(" ", ""))
    df["Date"] = pd.to_datetime(df["Date"])
    df["Expiry Date"] = pd.to_datetime(df["Expiry Date"])
    df = df[df["Date"] >= start_date]
    df = df[df["Date"] < end_date]
    backtester.parseOptionData(df, ticker, sampleOption=100)

register_option("/Users/andriylevitskyy/Desktop/tofina/tests/data/options/AAPL.csv", "AAPL")
register_option("/Users/andriylevitskyy/Desktop/tofina/tests/data/options/NVDA.csv", "NVDA")

dir_path = Path("./results/DeepAR_Options")
if dir_path.exists():
    shutil.rmtree(dir_path)
dir_path.mkdir(parents=True, exist_ok=False)
backtester.optimizeStrategy(dir_path, RiskAversion=0.9, stop=False, iterations = 5000)
backtesterCopy = deepcopy(backtester)
results = backtester.evaluateStrategy()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/opt/anaconda3/lib/python3.8/site-packages/pytorch_forecasting/data/samplers.py:86: UserWarning: Less than 32 samples available for 5486 prediction times. Use batch size smaller than 32. First 10 prediction times with small batch sizes: [31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is alread

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/50 [00:00<?, ?it/s]                            

/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 50/50 [00:20<00:00,  2.47it/s, v_num=3, train_loss_step=4.680, val_loss=4.380, train_loss_epoch=4.700]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 50/50 [00:20<00:00,  2.46it/s, v_num=3, train_loss_step=4.680, val_loss=4.380, train_loss_epoch=4.700]


/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
<

In [3]:
for ts in timestamps:
    print(ts, float(results[ts]["real"].sum(axis=1)), float(results[ts]["simulation"].sum(axis=1).mean()))


2023-01-03 00:00:00 -0.97748969898669 11156.4921875


In [4]:
timestamps =  list(backtester.pointInTimePortfolio.keys())
testPortfolio = backtester.pointInTimePortfolio[timestamps[0]]

In [5]:
weights = backtester.pointInTimePortfolio[timestamps[0]].strategy.normalizedWeights
instruments = backtester.pointInTimePortfolio[timestamps[0]].strategy.instruments.keys()
prices = []
for i in instruments:
    prices.append(backtester.pointInTimePortfolio[timestamps[0]].strategy.instruments[i].price)
weightsDict = {}
for w,i,p in zip(weights,instruments, prices):
    weightsDict[i] = {"weight":float(w), "price":float(p)}
pd.DataFrame(weightsDict).T.sort_values(by="weight", ascending=False)

,weight,price
NVDA_Put_113.0_4_NVDA,0.987076,0.010000
AAPL_Put_114.0_9_AAPL,0.000267,0.660000
NVDA_Put_197.5_19_NVDA,0.000260,56.500000
NVDA_Call_134.0_9_NVDA,0.000259,11.950000
AAPL_Call_136.0_19_AAPL,0.000259,1.430000
...,...,...
AAPL_Call_146.0_14_AAPL,0.000003,0.150000
NVDA_Call_197.5_19_NVDA,0.000003,0.150000
NVDA_Call_205.0_4_NVDA,0.000003,0.010000
NVDA_NVDA_Stock_NVDA,0.000002,143.149994
